<a href="https://colab.research.google.com/github/QuasimRizvi/BigData/blob/main/BigData_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [51]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [50]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [52]:
import findspark
findspark.init()

In [53]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [54]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-03-31 03:45:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.171.215.128, 3.226.107.193, 52.6.49.248, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.171.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.06M  63.3MB/s    in 0.2s    

2021-03-31 03:45:19 (63.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14746350/14746350]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  tes

In [55]:
ls

adc.json                      spark-3.1.1-bin-hadoop2.7.tgz
ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7.tgz.1
sample_data/                  spark-3.1.1-bin-hadoop2.7.tgz.2
spark-3.1.1-bin-hadoop2.7/


In [56]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col

In [57]:
project_id = 'api3-307502'

In [58]:
from google.colab import auth
auth.authenticate_user()

In [59]:
!gcloud config set project {project_id}

Updated property [core/project].


In [60]:
!gsutil cp gs://ethereum-bigdata/contracts* /tmp/

Copying gs://ethereum-bigdata/contracts_2019_11_01__2020_01_01_000000000000...
Copying gs://ethereum-bigdata/contracts_2019_11_01__2020_01_01_000000000001...
Copying gs://ethereum-bigdata/contracts_2019_11_01__2020_01_01_000000000002...
Copying gs://ethereum-bigdata/contracts_2020_03_01__2020_05_01_000000000000...
- [4 files][  1.6 GiB/  1.6 GiB]   48.7 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://ethereum-bigdata/contracts_2020_03_01__2020_05_01_000000000001...
Copying gs://ethereum-bigdata/contracts_2020_03_01__2020_05_01_000000000002...
Copying gs://ethereum-bigdata/contracts_2020_07_01__2020_09_01_000000000000...
Copying gs://ethereum-bigdata/contracts_2020_07_01__2020_09_01_000000000001...
Copying gs://ethereum-bigdata/contr

In [62]:
spark = SparkSession(sc)

In [67]:
contracts = os.listdir('/tmp')

In [68]:
contracts = ["/tmp/" +x for x in contracts if "contracts_" in x]
contracts.sort()
contracts

['/tmp/contracts_2019_11_01__2020_01_01_000000000000',
 '/tmp/contracts_2019_11_01__2020_01_01_000000000001',
 '/tmp/contracts_2019_11_01__2020_01_01_000000000002',
 '/tmp/contracts_2020_03_01__2020_05_01_000000000000',
 '/tmp/contracts_2020_03_01__2020_05_01_000000000001',
 '/tmp/contracts_2020_03_01__2020_05_01_000000000002',
 '/tmp/contracts_2020_07_01__2020_09_01_000000000000',
 '/tmp/contracts_2020_07_01__2020_09_01_000000000001',
 '/tmp/contracts_2020_07_01__2020_09_01_000000000002',
 '/tmp/contracts_2020_07_01__2020_09_01_000000000003',
 '/tmp/contracts_2020_11_01__2021_01_01_000000000000',
 '/tmp/contracts_2020_11_01__2021_01_01_000000000001',
 '/tmp/contracts_2020_11_01__2021_01_01_000000000002']

In [69]:
from functools import reduce
dfContracts = reduce(lambda x,y: x.unionAll(y), 
            [spark.read.format('com.databricks.spark.csv')
                       .load(f, header="true", inferSchema="true") 
             for f in contracts])

In [71]:
dfContracts.createOrReplaceTempView("contracts")

In [72]:
spark.sql("Select COUNT(*) FROM contracts").show()

+--------+
|count(1)|
+--------+
|10134812|
+--------+



In [73]:
!gsutil cp gs://ethereum-bigdata/transactions* /tmp/

Copying gs://ethereum-bigdata/transactions_2019_11_01__2020_01_01_000000000000...
Copying gs://ethereum-bigdata/transactions_2019_11_01__2020_01_01_000000000001...
Copying gs://ethereum-bigdata/transactions_2019_11_01__2020_01_01_000000000002...
Copying gs://ethereum-bigdata/transactions_2019_11_01__2020_01_01_000000000003...
/ [4 files][  1.8 GiB/  1.8 GiB]   49.2 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://ethereum-bigdata/transactions_2019_11_01__2020_01_01_000000000004...
Copying gs://ethereum-bigdata/transactions_2019_11_01__2020_01_01_000000000005...
Copying gs://ethereum-bigdata/transactions_2019_11_01__2020_01_01_000000000006...
Copying gs://ethereum-bigdata/transactions_2019_11_01__2020_01_01_000000000007...
Copying gs:

In [74]:
transactions = os.listdir('/tmp')
transactions = ["/tmp/" +x for x in transactions if "transactions_" in x]
transactions.sort()
transactions

['/tmp/transactions_2019_11_01__2020_01_01_000000000000',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000001',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000002',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000003',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000004',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000005',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000006',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000007',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000008',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000009',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000010',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000011',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000012',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000013',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000014',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000015',
 '/tmp/transactions_2019_11_01__2020_01_01_000000000016',
 '/tmp/transac

In [75]:
dfTransactions = reduce(lambda x,y: x.unionAll(y), 
            [spark.read.format('com.databricks.spark.csv')
                       .load(f, header="true", inferSchema="true") 
             for f in transactions])

In [76]:
dfTransactions.createOrReplaceTempView("transactions")
spark.sql("SELECT COUNT(*) FROM transactions").show()

+---------+
| count(1)|
+---------+
|221484978|
+---------+



In [77]:
dfContracts.printSchema()

root
 |-- address: string (nullable = true)
 |-- bytecode: string (nullable = true)
 |-- is_erc20: boolean (nullable = true)
 |-- is_erc721: boolean (nullable = true)
 |-- block_timestamp: string (nullable = true)
 |-- block_number: integer (nullable = true)



In [78]:
dfTransactions.printSchema()

root
 |-- hash: string (nullable = true)
 |-- nonce: integer (nullable = true)
 |-- transaction_index: integer (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- value: decimal(24,0) (nullable = true)
 |-- gas: integer (nullable = true)
 |-- gas_price: long (nullable = true)
 |-- receipt_cumulative_gas_used: integer (nullable = true)
 |-- receipt_gas_used: integer (nullable = true)
 |-- receipt_contract_address: string (nullable = true)
 |-- receipt_root: string (nullable = true)
 |-- receipt_status: integer (nullable = true)
 |-- block_timestamp: string (nullable = true)
 |-- block_number: integer (nullable = true)

